# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [103]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json 

# Import API key
from api_keys import geoapify_key

In [104]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachnyy,66.4167,112.4000,235.62,73,98,1.14,RU,1675818096
1,1,jamestown,42.0970,-79.2353,281.92,75,100,6.17,US,1675817815
2,2,nanortalik,60.1432,-45.2371,257.85,72,8,8.65,GL,1675818107
3,3,tukrah,32.5341,20.5791,284.52,65,99,4.81,LY,1675818108
4,4,half moon bay,37.4636,-122.4286,289.40,67,0,6.69,US,1675818108


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [105]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    scale = 1,
    size ="Humidity",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [106]:
# Narrow down cities that fit criteria and drop any results with null values
    
# Drop any rows with null values
city_data_clean = city_data_df.dropna()

# Itterate thorugh city temperatures 
city = [] 

for index, row in city_data_clean.iterrows():
    if row ["Max Temp"] <= 275:
        if row ["Cloudiness"] <25:
            if row ["Wind Speed"] <=5:
                if row["Humidity"] <=60:
                    city.append(row)
                
city_ideal_weather_df = pd.DataFrame(city)
   
# Display sample data
city_ideal_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
117,117,port arthur,38.8000,121.2667,271.27,45,0,4.48,CN,1675818275
146,146,severo-kurilsk,50.6789,156.1250,269.05,54,12,4.38,RU,1675818117
191,191,jumla,29.2747,82.1838,273.92,51,0,1.61,NP,1675818168
242,242,changping,40.2186,116.2197,271.95,46,5,1.87,CN,1675818414
267,267,mujiayingzi,42.1167,118.7833,265.83,34,0,2.64,CN,1675818360
310,310,cortez,37.3489,-108.5859,273.57,57,0,1.54,US,1675818480
324,324,glenwood springs,39.5505,-107.3248,272.23,51,0,0.00,US,1675818516
552,552,svetlaya,46.5389,138.3297,264.67,55,2,3.98,RU,1675818757


### Step 3: Create a new DataFrame called `hotel_df`.

In [107]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
117,port arthur,CN,38.8000,121.2667,45,
146,severo-kurilsk,RU,50.6789,156.1250,54,
191,jumla,NP,29.2747,82.1838,51,
242,changping,CN,40.2186,116.2197,46,
267,mujiayingzi,CN,42.1167,118.7833,34,
310,cortez,US,37.3489,-108.5859,57,
324,glenwood springs,US,39.5505,-107.3248,51,
552,svetlaya,RU,46.5389,138.3297,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [112]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params = {
    "apikey":geoapify_key
    "filter":filters,
    "bias":bias,
}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (3194883027.py, line 19)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [102]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 600,
    frame_length = 800,
    color = "City"
    )

# Display the map
# YOUR CODE HERE